In [1]:
import feedparser

d: feedparser.FeedParserDict = feedparser.parse(
    r"https://www.mavcsoport.hu/mavinform/rss.xml"
)
if d.status != 200:
    raise NotImplementedError("Handle if feed is down!")

In [2]:
print(d.feed.description, len(d.entries))
print(d.entries[0])

MÁVINFORM hírek 10
{'title': 'Prágából késik a 277-es Metropolitan', 'title_detail': {'type': 'text/plain', 'language': 'hu', 'base': 'https://www.mavcsoport.hu/mav-csoport/hirek', 'value': 'Prágából késik a 277-es Metropolitan'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.mavcsoport.hu/node/113787'}], 'link': 'https://www.mavcsoport.hu/node/113787', 'summary': '', 'summary_detail': {'type': 'text/html', 'language': 'hu', 'base': 'https://www.mavcsoport.hu/mav-csoport/hirek', 'value': ''}, 'published': 'Fri, 18 Nov 2022 16:07:30 +0100', 'published_parsed': time.struct_time(tm_year=2022, tm_mon=11, tm_mday=18, tm_hour=15, tm_min=7, tm_sec=30, tm_wday=4, tm_yday=322, tm_isdst=0), 'authors': [{'name': 'MÁV.hu szerkesztőség'}], 'author': 'MÁV.hu szerkesztőség', 'author_detail': {'name': 'MÁV.hu szerkesztőség'}, 'id': '113787', 'guidislink': False}


In [4]:
import os, sys

sys.path.append(os.path.dirname(os.path.realpath("../RSS_abstracts")))
from RSS_abstracts.RSSEntry import RSSEntry
import importlib

print(RSSEntry.__annotations__)

e = RSSEntry.from_dict(d.entries[0])
print(e)

{'id': <class 'int'>, 'title': <class 'str'>, 'link': <class 'str'>, 'published': <class 'str'>, 'published_parsed': typing.List[int], 'unparsed': typing.Dict}
RSSEntry(id='113787', title='Prágából késik a 277-es Metropolitan', link='https://www.mavcsoport.hu/node/113787', published='Fri, 18 Nov 2022 16:07:30 +0100', published_parsed=time.struct_time(tm_year=2022, tm_mon=11, tm_mday=18, tm_hour=15, tm_min=7, tm_sec=30, tm_wday=4, tm_yday=322, tm_isdst=0), unparsed={'title': 'Prágából késik a 277-es Metropolitan', 'title_detail': {'type': 'text/plain', 'language': 'hu', 'base': 'https://www.mavcsoport.hu/mav-csoport/hirek', 'value': 'Prágából késik a 277-es Metropolitan'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.mavcsoport.hu/node/113787'}], 'link': 'https://www.mavcsoport.hu/node/113787', 'summary': '', 'summary_detail': {'type': 'text/html', 'language': 'hu', 'base': 'https://www.mavcsoport.hu/mav-csoport/hirek', 'value': ''}, 'published': 'Fri, 18 Nov

In [12]:
import json

jstr = json.dumps(d.entries[0])
print(jstr)

{"title": "A Debrecen-Ny\u00edr\u00e1br\u00e1ny vonalon helyre\u00e1llhat a menetrend", "title_detail": {"type": "text/plain", "language": "hu", "base": "https://www.mavcsoport.hu/mav-csoport/hirek", "value": "A Debrecen-Ny\u00edr\u00e1br\u00e1ny vonalon helyre\u00e1llhat a menetrend"}, "links": [{"rel": "alternate", "type": "text/html", "href": "https://www.mavcsoport.hu/node/113301"}], "link": "https://www.mavcsoport.hu/node/113301", "summary": "", "summary_detail": {"type": "text/html", "language": "hu", "base": "https://www.mavcsoport.hu/mav-csoport/hirek", "value": ""}, "published": "Wed, 09 Nov 2022 11:56:06 +0100", "published_parsed": [2022, 11, 9, 10, 56, 6, 2, 313, 0], "authors": [{"name": "M\u00c1V.hu szerkeszt\u0151s\u00e9g"}], "author": "M\u00c1V.hu szerkeszt\u0151s\u00e9g", "author_detail": {"name": "M\u00c1V.hu szerkeszt\u0151s\u00e9g"}, "id": "113301", "guidislink": false}


In [ ]:
link = d.entries[0]["link"]
import requests

res = requests.get(link)
if res.status_code != requests.codes.OK:
    raise ValueError("Response Failed!")

In [ ]:
from bs4 import BeautifulSoup

parsed = BeautifulSoup(res.text, "html.parser")
print(parsed.title)

print(parsed.select("#block-system-main .content.clearfix"))
# print([x for x in parsed.select("#block-system-main .content.clearfix")[0].children ])
# print([x for x in parsed.select("#block-system-main .content.clearfix")[0].stripped_strings])

content = parsed.select("#block-system-main .content.clearfix")[0]

<title>A Zemplén InterCity Szerencstől késve közlekedik | MÁV-csoport</title>
[<div class="content clearfix">
<div class="node-date">2022. november 6. vasárnap, 19.39</div>
<div class="field-body">
<p>A Miskolc-Tiszai pályaudvarról Sátoraljaújhelyre 18:40-kor elindult <strong>Zemplén InterCity (526)</strong> mozdonya Szerencs állomáson meghibásodott, ezért <strong>várhatóan 40 perc késéssel indul tovább. </strong></p>
<p>MÁVINFORM</p>
</div>
</div>]


In [ ]:
k = parsed.select(".field-body")[0]
print(" ".join(k.stripped_strings))

A Miskolc-Tiszai pályaudvarról Sátoraljaújhelyre 18:40-kor elindult Zemplén InterCity (526) mozdonya Szerencs állomáson meghibásodott, ezért várhatóan 40 perc késéssel indul tovább. MÁVINFORM


In [ ]:
import importlib
import os, sys

sys.path.append(os.path.dirname(os.path.realpath("../parser")))
from parser.MAV.webnode_parser import WebNodeParser

In [ ]:
for e in d.entries:
    print(WebNodeParser.str_form_entry(e)[0:50])

A Miskolc-Tiszai pályaudvarról Sátoraljaújhelyre 1
Frissítés: Rákos és Pécel között már mindkét vágán
A Vámosgyörkről Szolnokra 5:22-kor elindult személ
Frissítés: Villány és Áta között már közlekednek a
Frissítés: Városlőd-Kislőd állomás és Szentgál köz
A Szegedről Békéscsabára 16:28-kor elindult 7706-o
A Pécsről a Keleti pályaudvarra tartó 815-ös számú
Frissítés: Taktaharkány és Hernádnémeti-Bőcs állom
Frissítés: Az esztergomi vonal Óbuda–Solymár közöt
Frissítés: A Déli pályaudvarról Tapolcára 13:05-ko


In [ ]:
# check if etag is enabled
last_etag = d2.etag if d2 is not None else d.etag
d2: feedparser.FeedParserDict = feedparser.parse(
    r"https://www.mavcsoport.hu/mavinform/rss.xml", etag=last_etag
)
print(d2.etag)
print(d2.status)
# etag is working

W/"1667761362-1"
304
